In [1]:
# imports
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

/Users/jcolamendy/python/tutorials/nlp-tutorials/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [3]:
llm = ChatOpenAI()

In [9]:
llm.invoke("What is an deep learning?")

AIMessage(content='Deep learning is a subset of machine learning that uses artificial neural networks to model and solve complex problems. It is a type of machine learning that involves algorithms that are capable of learning and making decisions on their own, without the need for human intervention. Deep learning is commonly used in areas such as image and speech recognition, natural language processing, and computer vision.', response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 13, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-85588d0d-f44f-410c-a7d1-e71225c17b45-0')

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("user", "{text}")
])

In [5]:
output = prompt.format_prompt(input_language="English", output_language="French", text="I love programming.")
print(type(output))
print(output)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


In [6]:
chain = prompt | llm
output = chain.invoke({"input_language": "English", "output_language": "French", "text": "I love deep learning."})
print(type(output))
print(output)

<class 'langchain_core.messages.ai.AIMessage'>
content="J'adore l'apprentissage profond." response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 27, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None} id='run-fc95980e-1c79-4632-81e8-b4e979d5bc4a-0'


In [7]:
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

output = chain.invoke({"input_language": "English", "output_language": "French", "text": "I love programming."})
print(type(output))
print(output)

<class 'str'>
J'adore la programmation.


In [51]:
output_parser = StrOutputParser()
chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

output = chain.invoke({"input_language": "English", "output_language": "French", "text": "I love deep learning."})
print(type(output))
print(output)

<class 'dict'>
{'input_language': 'English', 'output_language': 'French', 'text': "J'adore l'apprentissage profond."}


In [9]:
# llm vs chat models
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.messages import HumanMessage

llm = OpenAI()
chat_model = ChatOpenAI()

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

output = chat_model.invoke(messages)
print(type(output))
print(output)

output = llm.invoke(text)
print(type(output))
print(output)

<class 'langchain_core.messages.ai.AIMessage'>
content='Rainbow Soles' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None} id='run-c6aef605-5710-4cca-96ed-1d940d1be3ea-0'
<class 'str'>


"Rainbow Socks Co." or "Spectrum Socks Inc."


In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
output = prompt.format(product="colorful socks")
print(type(output))
print(output)

<class 'str'>
What is a good name for a company that makes colorful socks?


In [17]:
from langchain_core.prompts import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

output = chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.")
print(type(output))
print(output)

output = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")
print(type(output))
print(output)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]
<class 'list'>
[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


In [18]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

['hi', 'bye']

In [19]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['blue', 'red', 'green', 'yellow', 'purple']

In [20]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


In [21]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import AIMessage, HumanMessage

human_prompt = "Summarize our conversation so far in {word_count} words."

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"), 
        HumanMessagePromptTemplate.from_template(human_prompt)
    ]
)

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count="10"
).to_messages()

[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

In [24]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")

output = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
print(output)
# StringPromptValue(text='Tell me a funny joke about chickens.')

output = prompt_val.to_string()
print(output)
#'Tell me a funny joke about chickens.'

output = prompt_val.to_messages()
print(output)
#[HumanMessage(content='Tell me a funny joke about chickens.')]

text='Tell me a funny joke about chickens.'
Tell me a funny joke about chickens.
[HumanMessage(content='Tell me a funny joke about chickens.')]


In [27]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
print(type(chat_val))
print(chat_val)

output = chat_val.to_messages()
print(output)

output = chat_val.to_string()
print(output)

<class 'langchain_core.prompt_values.ChatPromptValue'>
messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')]
[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')]
System: You are a helpful assistant that re-writes the user's text to sound more upbeat.
Human: i dont like eating tasty things.


In [28]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


In [29]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/05/2024, 00:14:29


In [32]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

model = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("user", "{text}")
])

chain = LLMChain(llm=model, prompt=prompt)
output = chain.run(input_language="English", output_language="French", text="I love programming")
print(type(output))
print(output)

<class 'str'>
J'adore la programmation.


In [33]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

model = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("user", "{text}")
])

chain = LLMChain(llm=model, prompt=prompt)
output = chain.invoke({"input_language": "English", "output_language": "French", "text": "I love programming."})
print(type(output))
print(output)

<class 'dict'>
{'input_language': 'English', 'output_language': 'French', 'text': "J'adore la programmation."}


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]
chat.invoke(messages)

AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model learns the training data too well, including noise and random fluctuations, which can lead to poor performance on new, unseen data. Regularization techniques add a penalty term to the model's loss function to discourage overly complex models that may fit the training data too closely. This helps to improve the model's generalization ability and make it more effective at making predictions on new data. Regularization techniques help strike a balance between model complexity and performance, leading to more robust and accurate models.", response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 24, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-c39b00d9-7643-4b63-aa8e-7f088647eddf-0')

In [7]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

The purpose of model regularization is to prevent a machine learning model from overfitting the training data. Overfitting occurs when a model learns the training data too well and performs poorly on new, unseen data. Regularization techniques help to control the complexity of the model by adding a penalty term to the loss function, which discourages overly complex models that may be fitting noise in the data rather than the underlying patterns. This helps improve the model's generalization performance on unseen data and makes it more robust. Common regularization techniques include L1 and L2 regularization, dropout, and early stopping.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model = llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

joke_chain = prompt | model | StrOutputParser()
# joke_chain.invoke({"topic": "bears"})

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")
analysis_chain = analysis_prompt | model | StrOutputParser()

composed_chain = {"joke": joke_chain} | analysis_chain

# call
composed_chain.invoke({"topic": "bears"})

'Some people may find this joke funny, while others may not. It ultimately depends on individual sense of humor.'

In [9]:
# another way
composed_chain_with_lambda = (
    joke_chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)
# call
composed_chain_with_lambda.invoke({"topic": "beets"})

'Yes, this joke is funny! It plays on the literal meaning of "root" as well as the common phrase "root of the problem."'

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

composed_chain_runnable = (
    RunnableParallel({"joke": joke_chain})
    | analysis_prompt
    | model
    | StrOutputParser()
)
# call
composed_chain_runnable.invoke({"topic": "battlestar galactica"})

'Some people may find this joke funny, especially if they are familiar with the TV show Battlestar Galactica and the relationship between Cylons and toasters. However, humor is subjective so not everyone may find it funny.'

In [17]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document

# docs
docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]
print(docs)
print(type(docs))

# embed docs into vector store
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# prompt
template = """You are an AI Assistant.

Your task is to answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
print(prompt)
print(type(prompt))

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain_input = {"context": retriever , "question": RunnablePassthrough()}
rag_chain = (
    chain_input
    | prompt
    | llm
    | StrOutputParser()
)

# call
rag_chain.invoke("What does Jesse love?")

print(rag_chain)
print(type(rag_chain))

[Document(page_content='Jesse loves red but not yellow'), Document(page_content='Jamal loves green but not as much as he loves orange')]
<class 'list'>
input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an AI Assistant.\n\nYour task is to answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))]
<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
first={
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x107993a90>),
  question: RunnablePassthrough()
} middle=[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an AI Assistant.\n\nYour task is to answer the question based only on the following context:\n{context}\n\nQuestion

In [18]:
from operator import itemgetter

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain_input = {
    "context": itemgetter("question") | retriever | format_docs, 
    #"question": RunnablePassthrough()
    "question": itemgetter("question")
}
rag_chain = (
    chain_input
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain)
print(type(rag_chain))

# call
rag_chain.invoke({"question": "What does Jesse love?"})

first={
  context: RunnableLambda(itemgetter('question'))
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x107993a90>)
           | RunnableLambda(format_docs),
  question: RunnableLambda(itemgetter('question'))
} middle=[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an AI Assistant.\n\nYour task is to answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))]), ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ee94fa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10fa65430>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')] last=StrOutputParser()
<class 'langchain_core.runnables.base.RunnableSequence'>


'Jesse loves red.'

In [16]:
retriever.invoke('What does Jamal love?')

[Document(page_content='Jamal loves green but not as much as he loves orange'),
 Document(page_content='Jamal loves green but not as much as he loves orange'),
 Document(page_content='Jesse loves red but not yellow'),
 Document(page_content='Jesse loves red but not yellow')]

In [4]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# prompt
qa_system_prompt = """You are an AI assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "Question: {input}"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo")

qa_chain = create_stuff_documents_chain(llm, qa_prompt)

# call
docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]
question = "What are everyone's favorite colors?"
ai_msg_1 = qa_chain.invoke({"input": question, "context": docs})
print(ai_msg_1)
print(type(ai_msg_1))

Jesse's favorite color is red, while Jamal's favorite color is orange, which he loves more than green. Jesse does not like yellow, and Jamal does not love green as much as he loves orange.
<class 'str'>


In [22]:
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

## create_stuff_documents_chain chain
# prompt
qa_system_prompt = """You are an AI assistant for question-answering tasks.
Your task is answer a question given the provided context by following the instructions below.

% INSTRUCTIONS:
You must follow the instructions:
- only use the context to answer the question
- if you don't know or find the answer, just say that you don't know
- use three sentences maximum for the answer
- keep the answer concise

% CONTEXT:
{context}"""

print(qa_system_prompt)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("human", "Question: {input}"),
    ]
)
print(type(qa_prompt))
print(qa_prompt)

You are an AI assistant for question-answering tasks.
Your task is answer a question given the provided context by following the instructions below.

% INSTRUCTIONS:
You must follow the instructions:
- only use the context to answer the question
- if you don't know or find the answer, just say that you don't know
- use three sentences maximum for the answer
- keep the answer concise

% CONTEXT:
{context}
<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['context', 'input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an AI assistant for question-answering tasks.\nYour task is answer a question given the provided context by following the instructions below.\n\n% INSTRUCTIONS:\nYou must follow the instructions:\n- only use the context to answer the question\n- if you don't know or find the answer, just say that you don't know\n- use three sentences maximum for the answer\n- keep the answer concise\n\n% 

In [23]:
# llm
llm = ChatOpenAI(model="gpt-3.5-turbo")

# build create_stuff_documents_chain chain
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
print(type(qa_chain))
print(qa_chain)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an AI assistant for question-answering tasks.\nYour task is answer a question given the provided context by following the instructions below.\n\n% INSTRUCTIONS:\nYou must follow the instructions:\n- only use the context to answer the question\n- if you don't know or find the answer, just say that you don't know\n- use three sentences maximum for the answer\n- keep the answer concise\n\n% CONTEXT:\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Question: {input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1297e2970>, async_client=<openai.resources

In [9]:
## retriever
# docs
docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

In [24]:
output = qa_chain.invoke({
    "context": docs,
    "input": "What are everyone's favorite colors?"
})
print(output)
print(type(output))

Jesse's favorite color is red, while Jamal's favorite color is orange. Jesse doesn't like yellow, and Jamal doesn't like green as much as he likes orange.
<class 'str'>


In [19]:
# embed docs into vector store
vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [25]:
## retrieval_chain chain
retrieval_chain = create_retrieval_chain(retriever, qa_chain)
print(type(retrieval_chain))
print(retrieval_chain)

<class 'langchain_core.runnables.base.RunnableBinding'>
bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x10ecba400>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are an AI assistant for question-answering tasks.\nYour task is answer a question given the provided context by following the instructions below.\n\n% INSTRUCTIONS:\nYou must follow the instructions:\n- only use the context to answer the question\n- if you 

In [26]:
retrieval_chain.invoke({"input": "What are everyone's favorite colors?"})

{'input': "What are everyone's favorite colors?",
 'context': [Document(page_content='Jesse loves red but not yellow'),
  Document(page_content='Jesse loves red but not yellow'),
  Document(page_content='Jesse loves red but not yellow'),
  Document(page_content='Jamal loves green but not as much as he loves orange')],
 'answer': "Jesse's favorite color is red, and Jamal's favorite color is orange. Jesse does not like yellow, and Jamal does not like green as much as he likes orange."}

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("user", "{text}")
])

chain = prompt | llm

print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input_language', 'output_language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='You are a helpful assistant that translates {input_language} to {output_language}.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x108ef0bb0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x108ef8640>, openai_api_key=SecretStr('**********'), openai_proxy='')


In [16]:
article = """
\
We believe AI's short—to mid-term future belongs to agents and that the long-term future of *AGI* may evolve from agentic systems. Our definition of agents covers any neuro-symbolic system in which we merge neural AI (such as an LLM) with semi-traditional software.

With agents, we allow LLMs to integrate with code — allowing AI to search the web, perform math, and essentially integrate into anything we can build with code. It should be clear the scope of use cases is phenomenal where AI can integrate with the broader world of software.

In this introduction to AI agents, we will cover the essential concepts that make them what they are and why that will make them the core of real-world AI in the years to come.

---

## Neuro-Symbolic Systems

Neuro-symbolic systems consist of both neural and symbolic computation, where:

- Neural refers to LLMs, embedding models, or other neural network-based models.
- Symbolic refers to logic containing symbolic logic, such as code.

Both neural and symbolic AI originate from the early philosophical approaches to AI: connectionism (now neural) and symbolism. Symbolic AI is the more traditional AI. Diehard symbolists believed they could achieve true AGI via written rules, ontologies, and other logical functions.

The other camp were the connectionists. Connectionism emerged in 1943 with a theoretical neural circuit but truly kicked off with Rosenblatt's perceptron paper in 1958 [1][2]. Both of these approaches to AI are fascinating but deserve more time than we can give them here, so we will leave further exploration of these concepts for a future chapter.

Most important to us is understanding where symbolic logic outperforms neural-based compute and vice-versa.

| Neural | Symbolic |
| --- | --- |
| Flexible, learned logic that can cover a huge range of potential scenarios. | Mostly hand-written rules which can be very granular and fine-tuned but hard to scale. |
| Hard to interpret why a neural system does what it does. Very difficult or even impossible to predict behavior. | Rules are written and can be understood. When unsure why a particular ouput was produced we can look at the rules / logic to understand. |
| Requires huge amount of data and compute to train state-of-the-art neural models, making it hard to add new abilities or update with new information. | Code is relatively cheap to write, it can be updated with new features easily, and latest information can often be added often instantaneously. |
| When trained on broad datasets can often lack performance when exposed to unique scenarios that are not well represented in the training data. | Easily customized to unique scenarios. |
| Struggles with complex computations such as mathematical operations. | Perform complex computations very quickly and accurately. |

Pure neural architectures struggle with many seemingly simple tasks.
"""

In [18]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI

openai_model = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.0, model=openai_model)

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    """You are an AI assistant that helps generate articles."""
)

# the user prompt is provided by the user, in this case however the only dynamic
# input is the article
user_prompt = HumanMessagePromptTemplate.from_template("""
% GOAL
You are tasked with creating a name for a article.
% INSTRUCTIONS
- The name should be based of the context of the article.
- Be creative, but make sure the names are clear, catchy, and relevant to the theme of the article.
- Only output the article name, no other explanation or text can be provided.
% ARTICLE: {article}""")

prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# chain
title_chain = (
    {"article": lambda x: x["article"]}
    | prompt
    | llm
    | {"article_title": lambda x: x.content}
)
article_title = title_chain.invoke({"article": article})
print(article_title)

# raw chain
raw_title_chain = (
    {"article": lambda x: x["article"]}
    | prompt
    | llm
)
article_title_msg = raw_title_chain.invoke({"article": article})
print(article_title_msg)

{'article_title': '"Bridging Minds: The Rise of Neuro-Symbolic AI Agents"'}
content='"Bridging Minds: The Rise of Neuro-Symbolic AI Agents"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 675, 'total_tokens': 692, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BDHrCRvLAGP6n8InXtI7iN5uTItWy', 'finish_reason': 'stop', 'logprobs': None} id='run-e5b3ee78-e855-457f-85d6-e9c638a07984-0' usage_metadata={'input_tokens': 675, 'output_tokens': 17, 'total_tokens': 692, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [20]:
user_prompt = HumanMessagePromptTemplate.from_template(
    """
%GOAL
You are tasked with creating a description for the article and article title given as context.
% INSTRUCTIONS:
- Output the SEO friendly article description. Do not output anything other than the description.
% CONTEXT:
The article is here for you to examine:
---
{article}
---

Here is the article title '{article_title}'.""")

prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# chain
description_chain = (
    {
        "article": lambda x: x["article"],
        "article_title": lambda x: x["article_title"]
    }
    | prompt
    | llm
    | {"summary": lambda x: x.content}
)

article_description = description_chain.invoke({
    "article": article,
    "article_title": article_title["article_title"]
})
article_description

{'summary': 'Explore the transformative potential of neuro-symbolic AI agents in "Bridging Minds: The Rise of Neuro-Symbolic AI Agents." This article delves into the integration of neural and symbolic systems, highlighting how these innovative agents can revolutionize real-world applications by merging the flexibility of neural networks with the precision of symbolic logic. Discover the essential concepts behind neuro-symbolic systems, their advantages, and the future of AI as we navigate the evolving landscape of artificial intelligence. Join us in understanding why these agents are poised to become the cornerstone of AI development in the coming years.'}

In [23]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(description=(
        "Constructive feedback on the original paragraph"
    ))

openai_model = "gpt-4o-mini"

# llm
llm = ChatOpenAI(temperature=0.0, model=openai_model)

# system prompt
system_prompt = SystemMessagePromptTemplate.from_template(
    """%GOAL: You are an AI assistant that helps generate articles."""
)
user_prompt = HumanMessagePromptTemplate.from_template(
    """%TASK:
You are tasked with creating a new paragraph for the article given as context. 
Follow the instructions below.

% INSTRUCTIONS:
- Choose one paragraph to review and edit. During your edit ensure you provide constructive feedback to the user so theycan learn where to improve their own writing.
- Provide the output in `json` format following the schema below\n{format_instructions}

% CONTEXT:
The article is here for you to examine:
---
{article}
---
""")

parser = PydanticOutputParser(pydantic_object=Paragraph)
prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt]).partial(format_instructions=parser.get_format_instructions())

chain = (
    {"article": lambda x: x["article"]}
    | prompt
    | llm
    | parser
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

chain.invoke({'article': article})

{'original_paragraph': 'Pure neural architectures struggle with many seemingly simple tasks.',
 'edited_paragraph': 'Pure neural architectures often encounter difficulties with tasks that may appear straightforward at first glance.',
 'feedback': "The original sentence is clear but could benefit from a slight rephrasing for improved clarity and flow. By using 'often encounter difficulties' instead of 'struggle,' the sentence becomes more formal and precise. Additionally, 'seemingly simple tasks' can be rephrased to 'tasks that may appear straightforward at first glance' to enhance readability and provide a smoother transition."}

In [27]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(description=(
        "Constructive feedback on the original paragraph"
    ))

openai_model = "gpt-4o-mini"

# llm
llm = ChatOpenAI(temperature=0.0, model=openai_model)
structured_llm = llm.with_structured_output(Paragraph)

# system prompt
system_prompt = SystemMessagePromptTemplate.from_template(
    """%GOAL: You are an AI assistant that helps generate articles."""
)
user_prompt = HumanMessagePromptTemplate.from_template(
    """%TASK:
You are tasked with creating a new paragraph for the article given as context. 
Follow the instructions below.

% INSTRUCTIONS:
- Choose one paragraph to review and edit. During your edit ensure you provide constructive feedback to the user so theycan learn where to improve their own writing.

% CONTEXT:
The article is here for you to examine:
---
{article}
---
""")

prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

chain = (
    {"article": lambda x: x["article"]}
    | prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

chain.invoke({'article': article})

{'original_paragraph': "Neuro-symbolic systems consist of both neural and symbolic computation, where: - Neural refers to LLMs, embedding models, or other neural network-based models. - Symbolic refers to logic containing symbolic logic, such as code. Both neural and symbolic AI originate from the early philosophical approaches to AI: connectionism (now neural) and symbolism. Symbolic AI is the more traditional AI. Diehard symbolists believed they could achieve true AGI via written rules, ontologies, and other logical functions. The other camp were the connectionists. Connectionism emerged in 1943 with a theoretical neural circuit but truly kicked off with Rosenblatt's perceptron paper in 1958 [1][2]. Both of these approaches to AI are fascinating but deserve more time than we can give them here, so we will leave further exploration of these concepts for a future chapter. Most important to us is understanding where symbolic logic outperforms neural-based compute and vice-versa.",
 'edi

In [32]:
system_prompt = SystemMessagePromptTemplate.from_template(
    """%GOAL: You are an AI assistant that helps generate articles."""
)
user_prompt = HumanMessagePromptTemplate.from_template(
    """%TASK:
You are tasked with creating a new paragraph for the article given as context. 
Follow the instructions below.

% INSTRUCTIONS:
- Choose one paragraph to review and edit. During your edit ensure you provide constructive feedback to the user so theycan learn where to improve their own writing.
- Provide the output in `json` format following the schema below\n{format_instructions}

% CONTEXT:
The article is here for you to examine:
---
{article}
---
""")

parser = PydanticOutputParser(pydantic_object=Paragraph)
prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt]).partial(format_instructions=parser.get_format_instructions())
chain = (
    {"article": lambda x: x["article"]}
    | prompt
    | llm
)
msg = chain.invoke({"article": article})
print(msg)

content='```json\n{\n  "original_paragraph": "Pure neural architectures struggle with many seemingly simple tasks.",\n  "edited_paragraph": "Pure neural architectures often encounter difficulties with tasks that may appear straightforward at first glance.",\n  "feedback": "The original sentence is clear but could benefit from a slight rephrasing to enhance its readability and flow. By using \'often encounter difficulties\' instead of \'struggle,\' the sentence becomes more formal and precise. Additionally, \'tasks that may appear straightforward at first glance\' adds a layer of nuance, suggesting that the simplicity of the task is subjective. Consider varying your sentence structure and using more descriptive language to engage the reader further."\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 911, 'total_tokens': 1049, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoni

In [33]:
print(type(msg))
out = parser.invoke(msg.content)
print(out)

<class 'langchain_core.messages.ai.AIMessage'>
original_paragraph='Pure neural architectures struggle with many seemingly simple tasks.' edited_paragraph='Pure neural architectures often encounter difficulties with tasks that may appear straightforward at first glance.' feedback="The original sentence is clear but could benefit from a slight rephrasing to enhance its readability and flow. By using 'often encounter difficulties' instead of 'struggle,' the sentence becomes more formal and precise. Additionally, 'tasks that may appear straightforward at first glance' adds a layer of nuance, suggesting that the simplicity of the task is subjective. Consider varying your sentence structure and using more descriptive language to engage the reader further."
